# Tool recommendation with Attention network 
## (Gated recurrent units Attention neural network with weighted cross-entropy loss)

In [9]:
import numpy as np
import json
import warnings
import operator

import tensorflow as tf
from keras import backend as K
import h5py

warnings.filterwarnings("ignore")


class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        self.units = units
        super(BahdanauAttention, self).__init__(**kwargs)

    def get_config(self):
        return super().get_config().copy()

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


class ToolPredictionAttentionModel():
  
    def __init__(self, parameters):
        self.embedding_size = int(parameters["embedding_size"])
        self.gru_units = int(parameters["gru_units"])
        self.max_len = parameters["max_len"]
        self.dimensions = parameters["dimensions"]
        self.learning_rate = parameters["learning_rate"]
        self.class_weights = parameters["class_weights"]
        self.spatial_dropout = parameters["spatial_dropout"]
        self.recurrent_dropout = parameters["recurrent_dropout"]
        self.dropout = parameters["dropout"]
        
    def weighted_loss(self, class_weights):
        """
        Create a weighted loss function. Penalise the misclassification
        of classes more with the higher usage
        """
        weight_values = list(class_weights.values())

        def weighted_binary_crossentropy(y_true, y_pred):
            # add another dimension to compute dot product
            expanded_weights = tf.keras.backend.expand_dims(weight_values, axis=-1)
            return tf.keras.backend.dot(tf.keras.backend.binary_crossentropy(y_true, y_pred), expanded_weights)
        return weighted_binary_crossentropy

    def create_model(self):
        sequence_input = tf.keras.layers.Input(shape=(self.max_len,), dtype='int32')
        embedded_sequences = tf.keras.layers.Embedding(self.dimensions, self.embedding_size, input_length=self.max_len, mask_zero=True)(sequence_input)
        embedded_sequences = tf.keras.layers.SpatialDropout1D(self.spatial_dropout)(embedded_sequences)
        gru_1 = tf.keras.layers.GRU(self.gru_units,
            return_sequences=True,
            return_state=False,
            activation='elu',
            recurrent_dropout=self.recurrent_dropout
        )
        gru_2 = tf.keras.layers.GRU(self.gru_units,
            return_sequences=True,
            return_state=True,
            activation='elu',
            recurrent_dropout=self.recurrent_dropout
        )
        gru_output = gru_1(embedded_sequences)
        gru_output = tf.keras.layers.Dropout(self.dropout)(gru_output)
        gru_output, gru_hidden = gru_2(gru_output)
        attention = BahdanauAttention(self.gru_units)
        context_vector, attention_weights = attention(gru_hidden, gru_output)
        dropout = tf.keras.layers.Dropout(self.dropout)(context_vector)
        output = tf.keras.layers.Dense(self.dimensions, activation='sigmoid')(dropout)
        model = tf.keras.Model(inputs=sequence_input, outputs=output)
        model.compile(
            optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate),
            loss=self.weighted_loss(self.class_weights),
        )
        return model


def load_model(model_path):
    trained_model = h5py.File(model_path, 'r')
    dictionary = json.loads(trained_model.get('data_dictionary').value)
    best_parameters = json.loads(trained_model.get('parameters').value)
    compatible_tools = json.loads(trained_model.get('compatible_tools').value)
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    new_model = ToolPredictionAttentionModel(best_parameters).create_model()
    new_model.load_weights(model_path)
    return dictionary, reverse_dictionary, best_parameters, compatible_tools, new_model


model_path = "data/tool_recommendation_attention_model.hdf5"
dictionary, reverse_dictionary, best_parameters, compatible_tools, new_model = load_model(model_path)

## Unpack trained model for prediction

In [10]:
print(reverse_dictionary)

{'1': 'term_id_vs_term_name', '2': 'naive_variant_caller', '3': 'rseqc_infer_experiment', '4': 'macs2_bdgcmp', '5': 'rseqc_inner_distance', '6': 'CONVERTER_bed_to_bgzip_0', '7': 'gatk2_variant_annotator', '8': 'EMBOSS: fuzzpro38', '9': 'bismark_bowtie2', '10': 'samtools_rmdup', '11': 'msconvert3_raw', '12': 'tp_multijoin_tool', '13': 'samtools_sort', '14': 'ctb_remIons', '15': 'gff2bed1', '16': 'secretbt2test', '17': 'iuc_pear', '18': 'Count1', '19': 'tophat2', '20': 'MSGFPlusAdapter', '21': 'bamFilter', '22': 'bedtools_mergebed', '23': 'vcftools_merge', '24': 'sam_merge2', '25': 'freebayes', '26': 'cshl_cut_tool', '27': 'bam_to_sam', '28': 'flexbar_split_RR_bcs', '29': 'rgPicardMarkDups', '30': 'macs2_callpeak', '31': 'CONVERTER_gff_to_bed_0', '32': 'deeptools_bamCompare', '33': 'antismash', '34': 'gtf_filter_by_attribute_values_list', '35': 'tp_find_and_replace', '36': 'MzTabExporter', '37': 'bg_uniq', '38': 'cshl_fastq_to_fasta', '39': 'gatk2_indel_realigner', '40': 'tp_split_on_col

In [11]:
def compute_recommendations(tool_sequence, labels, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    # predict next tools for a test path
    prediction = new_model.predict(sample_reshaped, verbose=0)
    class_weights = best_parameters["class_weights"]
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    prediction = np.reshape(prediction, (prediction.shape[1],))
    #prediction = prediction * weight_val
    prediction = prediction / float(np.max(prediction))
    prediction_pos = np.argsort(prediction, axis=-1)
    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_pos = [item for item in topk_prediction_pos if item != 0]
    topk_prediction_val = [int(prediction[pos] * 100) for pos in topk_prediction_pos]
    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        tool_name = reverse_dictionary[str(tool_pos)]
        pred_tool_ids_sorted[tool_name] = tool_pred_val
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    tool_seq_name = ",".join(tool_sequence_names)
    print("Current tool sequence: ")
    print()
    print(tool_seq_name)
    print()
    print("Recommended tools for the tool sequence '%s' with their scores in decreasing order:" % tool_seq_name)
    print()
    for i in pred_tool_ids_sorted:
        print(i + "(" + str(pred_tool_ids_sorted[i]) + "%)")
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids:")
    print()
    for i in ids_tools:
        print(i + "(" + str(ids_tools[i]) + ")")

## Indices of tools

In [12]:
topk = 20 # set the maximum number of recommendations
tool_seq = "2" # give tools ids in a sequence and see the recommendations. To know all the tool ids, 
                     # please print the variable 'reverse_dictionary'
compute_recommendations(tool_seq, "", topk)

Current tool sequence: 

naive_variant_caller

Recommended tools for the tool sequence 'naive_variant_caller' with their scores in decreasing order:

flexbar_split_RR_bcs(100%)
deeptools_profiler(96%)
eukaryotic_ncbi_submission(94%)
tp_awk_tool(92%)
Remove_ending(90%)
deeptools_correctGCBias(90%)
MSGFPlusAdapter(89%)
DatamashTranspose(88%)
deeptools_bigwigCompare(86%)
gatk2_variant_select(85%)
deeptools_computeGCBias(83%)
Datamash(79%)
silac_analyzer(79%)
bam2wig(79%)
cshl_cut_tool(78%)
bamCoverage_deepTools(77%)
deseq2_single(77%)
cshl_awk_replace_in_column(76%)
msconvert3_raw(74%)
heatmapper(70%)

Tool ids:

flexbar_split_RR_bcs(28)
deeptools_profiler(239)
eukaryotic_ncbi_submission(176)
tp_awk_tool(244)
Remove_ending(206)
deeptools_correctGCBias(69)
MSGFPlusAdapter(20)
DatamashTranspose(213)
deeptools_bigwigCompare(295)
gatk2_variant_select(308)
deeptools_computeGCBias(201)
Datamash(182)
silac_analyzer(214)
bam2wig(228)
cshl_cut_tool(26)
bamCoverage_deepTools(278)
deseq2_single(62)


## Recommended tools

In [13]:
print(new_model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 25, 132)           40920     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 25, 132)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 25, 122)           93696     
_________________________________________________________________
dropout_2 (Dropout)          (None, 25, 122)           0         
_________________________________________________________________
gru_3 (GRU)                  [(None, 25, 122), (None,  90036     
_________________________________________________________________
bahdanau_attention_1 (Bahdan ((None, 122), (None, 25,  3013

In [14]:
print(best_parameters)

{'embedding_size': 132, 'gru_units': 122, 'spatial1d_dropout': 0.4, 'dropout': 0.35000000000000003, 'recurrent_dropout': 0.35000000000000003, 'learning_rate': 0.03692115994017956, 'tuner/epochs': 1, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0, 'batch_size': 32, 'max_len': 25, 'dimensions': 310, 'class_weights': {'0': 0.0, '1': 0.09543005443725577, '2': 1.6582978932139152, '3': 4.812256047599217, '4': 0.6960840387669792, '5': 0.13845555899307133, '6': 1.2530738801573276, '7': 0.09538827675467865, '8': 0.07964582277129585, '9': 2.36302973908641, '10': 5.992613886364503, '11': 0.0, '12': 3.3953636085831604, '13': 4.556972389179466, '14': 0.07041739104367695, '15': 2.163323025660538, '16': 0.07250647687303469, '17': 0.5097738248193585, '18': 3.747675549429567, '19': 4.893910221874146, '20': 0.0, '21': 5.2482257831848935, '22': 4.387828804608451, '23': 2.3826786024540727, '24': 5.6680003185450945, '25': 6.576772652723754, '26': 0.0, '27': 0.0, '28': 0.0, '29': 0.588913574